# Needleman–Wunsch

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from IPython.display import display, Markdown, Latex

In [2]:
import sys
sys.path.insert(0, '..')

## Get Data

In [3]:
from phylo.main import load_snippets_data, load_snippets_md

In [4]:
data = load_snippets_data('mult')
md = load_snippets_md('mult')

# Using Algo

In [5]:
from phylo.needleman_wunsch import (
    nw, arr_to_frames, arr_to_table,
    traceback
)

In [6]:
s1 = "hello"
s2 = "hallo"

In [7]:
arr = nw(s1, s2)

In [8]:
arr_to_table(arr, s1, s2)

,,h,e,l,l,o
,[. 0],[← -1],[← -2],[← -3],[← -4],[← -5]
h,[↑ -1],[↖ 1],[← 0],[← -1],[← -2],[← -3]
a,[↑ -2],[↑ 0],[↖ 0],[↖ -1],[↖ -2],[↖ -3]
l,[↑ -3],[↑ -1],[↖ -1],[↖ 1],[↖ 0],[← -1]
l,[↑ -4],[↑ -2],[↖ -2],[↖ 0],[↖ 2],[← 1]
o,[↑ -5],[↑ -3],[↖ -3],[↑ -1],[↑ 1],[↖ 3]


In [9]:
a, b = traceback(arr['dir'], s1, s2)

In [10]:
def identity_score(aligned_s1, aligned_s2):
    """
    Get identity score for aligned strings
    """
    assert len(aligned_s1) == len(aligned_s2), "Strings must be the same length"
    
    identity = 0
    length = len(aligned_s1)
    
    for i in range(length):
        if a[i] == b[i]:
            identity += 1
            
    frac = f"{identity}/{length}"
    percent = identity / length
    
    return frac, percent

In [12]:
identity_score(a, b)

('4/5', 0.8)

In [13]:
def traceback(dir_matrix, s1, s2):
    row_idx = len(res) - 1
    col_idx = len(res[1]) - 1
    return _traceback_helper(res, row_idx, col_idx, s1, s2, "", "")


def _traceback_helper(res, row, col, s1, s2, str1, str2):
    cell = res[row, col]
    if cell == "D":
        str1 += s1[col - 1]
        str2 += s2[row - 1]
        return _traceback_helper(res, row - 1, col - 1, s1, s2, str1, str2)
    elif cell == "H":
        str1 += s1[col - 1]
        str2 += "-"
        return _traceback_helper(res, row, col - 1, s1, s2, str1, str2)
    elif cell == "V":
        str1 += "-"
        str2 += s2[row - 1]
        return _traceback_helper(res, row - 1, col, s1, s2, str1, str2)
    elif cell == ".":
        return str1[::-1], str2[::-1]
    else:
        print("error!")

In [25]:
letters = arr['dir']

In [26]:
traceback(letters, s1, s2)

('hello', 'hallo')

In [27]:
letters

array([['.', 'H', 'H', 'H', 'H', 'H'],
       ['V', 'D', 'H', 'H', 'H', 'H'],
       ['V', 'V', 'D', 'D', 'D', 'D'],
       ['V', 'V', 'D', 'D', 'D', 'H'],
       ['V', 'V', 'D', 'D', 'D', 'H'],
       ['V', 'V', 'D', 'V', 'V', 'D']], dtype='<U1')

## Algo

In [5]:
def arr_to_frames(arr):
    cols = [' '] + list(s1)
    idx = [' '] + list(s2)
    
    vals = pd.DataFrame(arr['val'], columns=cols, index=idx)
    dirs = (pd.DataFrame(arr['dir'], columns=cols, index=idx)
            .replace(regex="H", value='\u2190')
            .replace(regex="V", value='\u2191')
            .replace(regex="D", value='\u2196'))
    
    return vals, dirs

In [6]:
def see(arr):
    vals, dirs = arr_to_frames(arr)
    combined = "[" + dirs + " " + vals.astype(np.unicode_) + "]"
    display(combined)

In [7]:
s1 = "dood" # top
s2 = "dod"  # side

match = 1
mismatch = -1
gap = -1

In [11]:
penalty = {"match": match, "miss": mismatch, "gap": gap}

In [12]:
w = len(s1) + 1  # width
h = len(s2) + 1  # height

In [13]:
arr = np.empty(
    (h, w),
    dtype=[('val', 'i4'),('dir', np.unicode_, 1)]
)
arr.fill((0, '.'))  # 111 is a visible placeholder

In [14]:
see(arr)

,,d,o,o,d
,[. 0],[. 0],[. 0],[. 0],[. 0]
d,[. 0],[. 0],[. 0],[. 0],[. 0]
o,[. 0],[. 0],[. 0],[. 0],[. 0]
d,[. 0],[. 0],[. 0],[. 0],[. 0]


In [15]:
# fill in first row & col, descending

arr[0,0]['val'] = 0

arr[0,1:]['val'] = np.arange(-1, -w, -1)
arr[0,1:]['dir'] = 'H'

arr[1:,0]['val'] = np.arange(-1, -h, -1)
arr[1:,0]['dir'] = 'V'

In [16]:
see(arr)

,,d,o,o,d
,[. 0],[← -1],[← -2],[← -3],[← -4]
d,[↑ -1],[. 0],[. 0],[. 0],[. 0]
o,[↑ -2],[. 0],[. 0],[. 0],[. 0]
d,[↑ -3],[. 0],[. 0],[. 0],[. 0]


In [65]:
for i in range(1, h):
    for j in range(1, w):
        
        # determine val of horizontal, vertical, and diagnol option
        hor_val = arr[i, j-1]['val'] + penalty["gap"]
        vert_val = arr[i-1, j]['val'] + penalty["gap"]
        diag_val = arr[i-1, j-1]['val'] + (
            penalty["match"]
            if s1[j-1] == s2[i-1]
            else penalty['miss']
        )
        
        # find max value and its corresponding direction
        # (Note: currently ignores ties)
        dirs = {hor_val: 'H', vert_val: 'V', diag_val: 'D'}   
        selected_val = max(hor_val, vert_val, diag_val)
        selected_dir = dirs[selected_val]
        
        arr[i, j]['val'] = selected_val
        arr[i, j]['dir'] = selected_dir

In [66]:
see(arr)

,,d,o,o,d
,[. 0],[← -1],[← -2],[← -3],[← -4]
d,[↑ -1],[↖ 1],[← 0],[← -1],[↖ -2]
o,[↑ -2],[↑ 0],[↖ 2],[↖ 1],[← 0]
d,[↑ -3],[↖ -1],[↑ 1],[↖ 1],[↖ 2]


In [67]:
hor_val = 10
vert_val = 10
diag_val = 9

In [68]:
max(10, 10, 7)

10

In [69]:
dirs = {hor_val: 'H', vert_val: 'V', diag_val: 'D'}

In [70]:
dirs[10]

'V'

In [71]:
dirs

{10: 'V', 9: 'D'}